# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [35]:
import numpy as np
import pandas as pd

from gensim.models import Word2Vec 

from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2023-04-22 12:53:04.046316: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG6FEX7/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG6FEX7/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2023-04-22 12:53:29.890666: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure: 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [7]:
# YOUR CODE HERE
word2vec = Word2Vec(X_train, vector_size=100, window=5, min_count=5)
wv = word2vec.wv

Let's reuse the functions of the previous exercise to convert your training and test data into something you can feed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad`:
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from finding errors too late and from letting them propagate through the entire notebook.

In [10]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you are doing something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [16]:
# YOUR CODE HERE
np.unique(y_train, return_counts=True)[1]

array([1265, 1235])

In [23]:
baseline_accuracy = 0.5

# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [30]:
X_train_pad.shape

(2500, 200, 100)

In [37]:
# YOUR CODE HERE
model = Sequential()
model.add(layers.Masking(mask_value=0.0, input_shape=(200, 100)))
model.add(layers.LSTM(20, activation='tanh'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics = ['accuracy'])

❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy will greatly depend on your training corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [38]:
# YOUR CODE HERE
es = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train_pad, y_train, epochs=500, validation_split=0.2, callbacks=[es], verbose=1)

Epoch 1/500
63/63 [==============================] - 4s 41ms/step - loss: 0.6915 - accuracy: 0.5320 - val_loss: 0.6858 - val_accuracy: 0.5680
Epoch 2/500
63/63 [==============================] - 2s 30ms/step - loss: 0.6810 - accuracy: 0.5540 - val_loss: 0.6772 - val_accuracy: 0.5780
Epoch 3/500
63/63 [==============================] - 2s 30ms/step - loss: 0.6677 - accuracy: 0.5945 - val_loss: 0.6717 - val_accuracy: 0.5940
Epoch 4/500
63/63 [==============================] - 2s 30ms/step - loss: 0.6559 - accuracy: 0.6215 - val_loss: 0.6553 - val_accuracy: 0.6200
Epoch 5/500
63/63 [==============================] - 2s 30ms/step - loss: 0.6430 - accuracy: 0.6350 - val_loss: 0.6868 - val_accuracy: 0.5880
Epoch 6/500
63/63 [==============================] - 2s 30ms/step - loss: 0.6319 - accuracy: 0.6430 - val_loss: 0.6514 - val_accuracy: 0.6220
Epoch 7/500
63/63 [==============================] - 2s 30ms/step - loss: 0.6251 - accuracy: 0.6505 - val_loss: 0.6485 - val_accuracy: 0.6240
Epoch 

❓ **Question** ❓ Evaluate your model on the test set

In [39]:
# YOUR CODE HERE
model.evaluate(X_test_pad, y_test)

79/79 [==============================] - 1s 7ms/step - loss: 0.6309 - accuracy: 0.6592


[0.6308811902999878, 0.6592000126838684]

# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. Let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that others have learned? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this: 

In [40]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is smaller (65 MB).

</details>

In [41]:
# YOUR CODE HERE
word2vec_transfer = api.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [44]:
# YOUR CODE HERE
len(word2vec_transfer.key_to_index)

400000

In [45]:
word2vec_transfer.vector_size

50

❓ Let's embed `X_train` and `X_test`, same as in the first question where we provided the functions to do so! (There is a slight difference in the `embed_sentence_with_TF` function that we will not dig into)

In [46]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [47]:
# YOUR CODE HERE
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post')
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post')

❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training here could take some time. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [49]:
X_train_pad_2.shape

(2500, 1122, 50)

In [50]:
# YOUR CODE HERE
model = Sequential()
model.add(layers.Masking(mask_value=0.0, input_shape=(1122, 50)))
model.add(layers.LSTM(20, activation='tanh'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', 
              metrics = ['accuracy'])

In [54]:
es = EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train_pad_2, y_train, epochs=100, validation_split=0.2, callbacks=[es], verbose=1)

Epoch 1/100
63/63 [==============================] - 10s 155ms/step - loss: 0.1435 - accuracy: 0.9545 - val_loss: 0.7516 - val_accuracy: 0.7880
Epoch 2/100
63/63 [==============================] - 10s 153ms/step - loss: 0.1234 - accuracy: 0.9630 - val_loss: 0.7470 - val_accuracy: 0.7860
Epoch 3/100
63/63 [==============================] - 10s 153ms/step - loss: 0.1194 - accuracy: 0.9665 - val_loss: 0.7578 - val_accuracy: 0.7820
Epoch 4/100
63/63 [==============================] - 10s 153ms/step - loss: 0.1191 - accuracy: 0.9650 - val_loss: 0.8350 - val_accuracy: 0.7980
Epoch 5/100
63/63 [==============================] - 10s 153ms/step - loss: 0.1136 - accuracy: 0.9600 - val_loss: 0.8307 - val_accuracy: 0.7920
Epoch 6/100
63/63 [==============================] - 10s 157ms/step - loss: 0.1113 - accuracy: 0.9675 - val_loss: 0.8124 - val_accuracy: 0.7840
Epoch 7/100
63/63 [==============================] - 10s 151ms/step - loss: 0.0941 - accuracy: 0.9715 - val_loss: 0.9366 - val_accuracy:

In [55]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 78.800%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarded words that were not present more than a given number of times in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously

In [60]:
!git add 'sentiment-analysis-with-word2vec.ipynb'
!git commit -m 'Completed the notebook!'
!git push origin master

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	~/

nothing added to commit but untracked files present (use "git add" to track)
Everything up-to-date
